In [84]:
from functools import partial
import numpy as np 
import scipy

from pyscf import gto, scf

np.set_printoptions(8)
np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])  

In [ ]:
mol = gto.Mole()
# mol.atom = """
# O 0.00000000 0.00000000 0.10683000
# H 0.00000000 0.78517800 -0.42731900
# H 0.00000000 -0.78517800 -0.42731900
# """
mol.atom = """
Li 1.60 0.0 0.0 
H 0.0 0.0 0.0
"""
# mol.atom = """
# H 0.75 0.0 0.0 
# H 0.0 0.0 0.0
# """

mol.basis = "6-31G"
mol.verbose = 5
mol.build()
scf_eng = scf.RHF(mol)
scf_eng.conv_tol = 1e-6
scf_eng.conv_tol_grad = 1e-6
scf_eng.max_cycle = 100
scf_eng.kernel()
scf_eng.converged

System: uname_result(system='Linux', node='xps15-7590', release='5.15.167.4-microsoft-standard-WSL2', version='#1 SMP Tue Nov 5 00:21:55 UTC 2024', machine='x86_64')  Threads 4
Python 3.11.10 (main, Oct  3 2024, 07:29:13) [GCC 11.2.0]
numpy 1.26.0  scipy 1.14.1  h5py 3.12.1
Date: Sun Dec  8 18:37:10 2024
PySCF version 2.7.0
PySCF path  /home/zbwu/soft/miniconda3/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 5
[INPUT] max_memory = 4000 
[INPUT] num. atoms = 2
[INPUT] num. electrons = 4
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 Li     1.600000000000   0.000000000000   0.000000000000 AA    3.023561799304   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.000000000000   0.000000000000   0.000000000000 AA    0.0000000

True

In [107]:
def read_integral(filename):
    res = []
    with open(filename) as f:
        for line in f:
            if len(line) != 0:
                res.append(float(line))
    return np.array(res)

sorb = 11

s_ovlp = read_integral("./integral/S_LiH.txt").reshape(sorb, sorb)
h1e = read_integral("./integral/Hcore_LiH.txt").reshape(sorb, sorb)
h2e = read_integral("./integral/LiH.txt").reshape(sorb, sorb, sorb, sorb)

# s_ovlp = read_integral("./integral/S_H2.txt").reshape(sorb, sorb)
# h1e = read_integral("./integral/Hcore_H2.txt").reshape(sorb, sorb)
# h2e = read_integral("./integral/H2.txt").reshape(sorb, sorb, sorb, sorb)


s_ovlp_1 = mol.intor("int1e_ovlp")
h1e_1 = mol.intor("int1e_nuc") + mol.intor("int1e_kin")
h2e_1 = mol.intor("int2e")

print(np.abs(h1e - h1e_1).sum())
print(np.abs(h2e - h2e).sum())
print(np.abs(s_ovlp - s_ovlp).sum())

7.478212834827885e-07
0.0
0.0


In [108]:
def write_integral(filename):
    with open(filename, "w") as f:
        for i in range(len(h1e_1.reshape(-1))):
            f.write(f"{h1e_1.reshape(-1)[i]:.18f} \n")

# write_integral("./integral/Hcore_H2-pyscf.txt")
write_integral("./integral/Hcore_LiH-pyscf.txt")

In [112]:
s_ovlp = mol.intor("int1e_ovlp")
h1e = mol.intor("int1e_nuc") + mol.intor("int1e_kin")
h2e = mol.intor("int2e")
# print(mol.intor("int2e_spinor").shape, h1e.shape)
# X = np.linalg.inv(np.linalg.cholesky(S).T)

print(f"ovlp: {s_ovlp}")
natm = mol.natm
nmo = nao = mol.nao
nocc = mol.nelec[0] # 占据轨道数
so = slice(0, nocc)
# μ(mu), ν(nu), κ(kappa), λ(lambda), σ(sigma)为原子轨道
# i, j, k, j 为分子轨道
# a, b, c, d 为非占据轨道
# p, q, r, s, m  全分子轨道
# print(s_ovlp)
# print(h1e)


X = scipy.linalg.fractional_matrix_power(s_ovlp, -0.5)

import torch
value, Vector = torch.linalg.eigh(torch.from_numpy(s_ovlp))
s_diag = torch.diag(value**(-0.5))
# print(s_diag)
# Vector @ s_diag.numpy() @ Vector.T , X
X = Vector @ s_diag.numpy() @ Vector.T

ovlp: [[ 1.          0.14525823  0.18574248  0.          0.          0.
   0.          0.          0.          0.01994962  0.1352232 ]
 [ 0.14525823  1.          0.90551575  0.          0.          0.
   0.          0.          0.          0.24462069  0.56222874]
 [ 0.18574248  0.90551575  1.          0.          0.          0.
   0.          0.          0.          0.19770631  0.5186447 ]
 [ 0.          0.          0.          1.          0.          0.
   0.80206391  0.          0.         -0.35376678 -0.63611435]
 [ 0.          0.          0.          0.          1.          0.
   0.          0.80206391  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          1.
   0.          0.          0.80206391  0.          0.        ]
 [ 0.          0.          0.          0.80206391  0.          0.
   1.          0.          0.         -0.215529   -0.48631845]
 [ 0.          0.          0.          0.          0.80206391  0.
   0.          1.        

In [113]:
#calculate nucleus energy 
Z_A = mol.atom_charges()
r_A = mol.atom_coords()

r_AB = np.zeros((mol.natm, mol.natm))
for i in range(mol.natm):
    for j in range(mol.natm):
        if i != j:
            r_AB[i, j] = np.linalg.norm(r_A[i] - r_A[j])
        else:
            r_AB[i, j] = np.infty 
E_nuc = 0.5 * np.einsum("A, B , AB ->", Z_A, Z_A, 1/r_AB)
print(f"Nucleus energy is  {E_nuc:8f} {mol.energy_nuc():8f} a.u. ")

Nucleus energy is  0.992207 0.992207 a.u. 


In [ ]:
D = h1e
# print(h1e)
D_old = np.zeros((nao, nao))
count = 0 

print(D)

coeff_matrix = np.zeros((nao, nao))
# while (not np.allclose(D, D_old)):
while (True and count < 20):
    # if count > 3:
    #     raise ValueError("SCF does not converge")

    count += 1
    D_old = D
    # print(f"D_old\n: {D}")
    # Fock F_uv = H_uv + P_kl(uv|kl) - 0.5 *P_kl(ul|kv)
    fock_matrix = h1e + np.einsum("uvkl, kl ->uv", h2e, D) - 0.5 * np.einsum("ulkv, kl -> uv", h2e, D) 
    #Fock^{prime} F'= X.T @ F @ X 
    # print(f"fock:\n {fock_matrix}")
    
    # print(f"X:\n {X}")
    # break
    
    fock_matrix_p = X.T @ fock_matrix @ X
    
    # F'C' = C'e 
    eig_val, eig_vec = np.linalg.eigh(fock_matrix_p)
    coeff_matrix = X @ eig_vec
    # print(f"C_new:\n {coeff_matrix}")
    
    # D_{uv} = 2C_{ui}C_{vi}
    # D = 2 * coeff_matrix[:, so] @ coeff_matrix[:, so].T  
    D = 2 * np.einsum("ui, vi->uv", coeff_matrix[:, so], coeff_matrix[:, so])
    E_elec = (h1e * D).sum() + 0.5 * np.einsum("uvkl, uv, kl ->", h2e, D, D) - 0.25 * np.einsum("ulkv, uv, kl ->", h2e, D, D)
    E_elec1 = np.einsum("vu, uv->", D, (h1e + fock_matrix)) * 0.5

    print(f'E_elec: {E_elec:.10f}, E_elec1: {E_elec1:.10f}')
    # print(f"D-new:\n {D}")


# E_elec = 0.5 *\sum_u\sum_v P_vu(H_uv + F_uv)
# F_uv = H_uv + + P_kl(uv|kl) - 0.5 *P_kl(ul|kv)
E_elec = (h1e * D).sum() + 0.5 * np.einsum("uvkl, uv, kl ->", h2e, D, D) - 0.25 * np.einsum("ulkv, uv, kl ->", h2e, D, D)
E_tot = E_elec + E_nuc 

print(coeff_matrix.shape)
print(coeff_matrix[:, so].shape)

print(f"SCF Converged in  {count} loops")
print(f"Electronic energy {E_elec} a.u.")
print(f"Total energy     {E_tot} a.u.")

print(f"Pyscf energy is {scf_eng.e_tot} a.u. ")

[[-4.77600495e+00 -5.76900291e-01 -7.96034846e-01  1.43706829e-02
   0.00000000e+00  0.00000000e+00  3.92917741e-03  0.00000000e+00
   0.00000000e+00 -8.87228545e-02 -5.86022749e-01]
 [-5.76900291e-01 -1.26311890e+00 -1.11548474e+00  1.21256951e-01
   0.00000000e+00  0.00000000e+00  7.64864256e-02  0.00000000e+00
   0.00000000e+00 -4.78968292e-01 -8.53737264e-01]
 [-7.96034846e-01 -1.11548474e+00 -1.10146660e+00  1.06862092e-01
   0.00000000e+00  0.00000000e+00  7.91503955e-02  0.00000000e+00
   0.00000000e+00 -3.92562230e-01 -7.70784792e-01]
 [ 1.43706829e-02  1.21256951e-01  1.06862092e-01 -1.11557965e+00
   0.00000000e+00  0.00000000e+00 -7.69391473e-01  0.00000000e+00
   0.00000000e+00  6.61036237e-01  8.56067810e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.02158201e+00  0.00000000e+00  0.00000000e+00 -7.06849885e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -